In [1]:
#Imports
import pandas as pd
import numpy as np
from numpy import delete
import scipy.stats as stats

#Código necesario para correr funciones desde otro archivo .ipynb de la misma carpeta
%run Preparar_datos.ipynb
%run TablaResumenCC.ipynb

# Índice: Qué hay en el documento
¿Las oscilaciones diarias vienen del exterior?
- CCs sin suavizar datos de Te-Th y Th-Tc
- CCs con los datos suavizados de Te-Th y Th-Tc

Significado de las abreviaturas:

- CC: Coeficientes de correlación
- Tc: Temperatura del Cruce (ºC)
- Te: Temperatura del exterior (ºC)
- Th: Temperatura de la entrada, también llamada Hall (ºC)

Solo obtener para los mese con oscilaciones claras:
- 1) Los meses con oscilaciones claras para el C02 del cruce son: 
 Julio, agosto y septiembre de 2021.
 Junio, julio, agosto y septiembre de 2022
- 2) Los meses con oscilaciones claras para el CO2 del hall son:
Julio, agosto, septiembre y octubre de 2021. Mayo, junio y julio de 2022

# 2.1) ¿Las oscilaciones diarias vienen del exterior? Coeficientes a partir de las temperaturas sin suavizar

### Preparar datos:

In [2]:
#Obtener los datos por semanas Cruce:
#2021
SemanaCruceJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx")
SemanaCruceAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")
SemanaCruceOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx")
#2022
SemanaCruceJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx")
SemanaCruceAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")

#Obtener los datos por semanas Exterior:
#2021
SemanaExteriorJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto21 = SepararPorSemanas( "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre21 = SepararPorSemanas( "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")
SemanaExteriorOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx")
#2022
SemanaExteriorJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")

#Obtener los datos por semanas Hall:
#2021
SemanaHallJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx")
SemanaHallAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx")
SemanaHallSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx")
SemanaHallOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx")
#2022
SemanaHallJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx")
SemanaHallAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx")
SemanaHallSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx")

### Obtener coeficientes:

In [3]:
def cc_Th_Tc_Te(DataframeCruce, DataframeExterior, DataframeHall):#Función adaptada para variables sin suavizar
    """
    Atributos:
    1) DataframeCruce: Lista de dataframes por semanas correspondiente a los datos de un mes de Cruce
    2) DataframeExterior: Lista de dataframes por semanas correspondiente a los datos de un mes de Exterior
    3) DataframeHall: Lista de dataframes por semanas correspondiente a los datos de un mes de Hall
    
    Devuelve:
    Una lista de listas, r. Cada elemento de r, r[i] con i=0, 1, 2, 3, 4, es una lista con los coeficiente de correlación y p 
    valores correspondientes a una semana.  r[i] = r_TD_Th, p_TD_Th, r_TD_Te, p_TD_Te, r_Th_Te, p_Th_Te
    
    ¿A qué corresponde cada elemento de r[i]?
    1) r_TD_Th: Coeficiente de correlación entre la temperatura del Cruce y del Hall
    2) p_TD_Th: P-valor correspondiente a r_TD_Th
    3) r_TD_Te: Coeficiente de correlación entre la temperatura del Cruce y la exterior
    4) p_TD_Te: P-valor correspondiente a r_TD_Te
    5) r_Th_Te: Coeficiente de correlación entre la temperatura del Hall y la exterior
    6) p_Th_Te: P-valor correspondiente a r_Th_Te
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeExterior[i].reset_index(drop=True, inplace=True)
        DataframeHall[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Th
        for index, _ in DataframeExterior[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]["Temperatura D (° C)"][index]),
                                  pd.isna(DataframeHall[i]['Temperatura A (° C)'][index]),
                                  pd.isna(DataframeExterior[i]["Tr Exterior (C Tr)"][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeExterior[i].drop([index], axis=0, inplace=True)
                DataframeHall[i].drop([index], axis=0, inplace=True)

        #Mediante scipy por semanas:
        r_TD_Th, p_TD_Th = stats.pearsonr(DataframeCruce[i]["Temperatura D (° C)"], DataframeHall[i]['Temperatura A (° C)'])
        r_TD_Te, p_TD_Te = stats.pearsonr(DataframeCruce[i]["Temperatura D (° C)"], DataframeExterior[i]["Tr Exterior (C Tr)"])
        r_Th_Te, p_Th_Te = stats.pearsonr(DataframeHall[i]['Temperatura A (° C)'], DataframeExterior[i]["Tr Exterior (C Tr)"])
        rs = r_TD_Th, p_TD_Th, r_TD_Te, p_TD_Te, r_Th_Te, p_Th_Te
        r.append(rs)
        i=i+1
    return(r)

In [4]:
#Obtener las correlaciones entre la temperatura del Cruce, la exterior y la del Hall:
#2021
r7_2021 = cc_Th_Tc_Te(SemanaCruceJulio21, SemanaExteriorJulio21, SemanaHallJulio21)
r8_2021 = cc_Th_Tc_Te(SemanaCruceAgosto21, SemanaExteriorAgosto21, SemanaHallAgosto21)
r9_2021 = cc_Th_Tc_Te(SemanaCruceSeptiembre21, SemanaExteriorSeptiembre21, SemanaHallSeptiembre21)
r10_2021 = cc_Th_Tc_Te(SemanaCruceOctubre21, SemanaExteriorOctubre21, SemanaHallOctubre21)
#2022
r7_2022 = cc_Th_Tc_Te(SemanaCruceJulio22, SemanaExteriorJulio22, SemanaHallJulio22)     
r8_2022 = cc_Th_Tc_Te(SemanaCruceAgosto22, SemanaExteriorAgosto22, SemanaHallAgosto22)
r9_2022 = cc_Th_Tc_Te(SemanaCruceSeptiembre22, SemanaExteriorSeptiembre22, SemanaHallSeptiembre22)

In [5]:
#Representar los valores de los coeficientes y p-valores en una tabla:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['r_TD_Th', 'p_TD_Th', 'r_TD_Te', 'p_TD_Te', 'r_Th_Te', 'p_Th_Te']
display(CoefCorrel)
CoefCorrel.describe()

,r_TD_Th,p_TD_Th,r_TD_Te,p_TD_Te,r_Th_Te,p_Th_Te
Sem1 Julio 2021,-0.118979,1.969026e-02,-0.446509,3.249592e-20,0.729325,6.007110e-65
Sem2 Julio 2021,-0.497148,3.065543e-43,-0.636375,1.425036e-77,0.452203,3.528753e-35
Sem3 Julio 2021,-0.389129,1.016272e-25,-0.502621,2.633099e-44,0.744947,6.832452e-120
Sem4 Julio 2021,0.403816,9.564037e-28,0.216960,1.334596e-08,0.176380,4.227383e-06
Sem5 Julio 2021,0.367915,6.658926e-20,-0.253735,6.499979e-10,0.235932,9.973862e-09
Sem1 Agosto 2021,0.008298,9.360466e-01,-0.069133,5.033092e-01,0.904865,1.236904e-36
Sem2 Agosto 2021,0.725367,9.225323e-111,0.086891,2.428848e-02,0.374786,7.752015e-24
Sem3 Agosto 2021,0.843696,6.586904e-183,0.444237,8.008983e-34,0.430670,1.120188e-31
Sem4 Agosto 2021,0.754999,6.517387e-125,0.659492,4.198765e-85,0.771232,1.481650e-133
Sem5 Agosto 2021,0.548366,2.226065e-27,0.633956,1.313006e-38,0.634553,1.064943e-38


,r_TD_Th,p_TD_Th,r_TD_Te,p_TD_Te,r_Th_Te,p_Th_Te
count,35.000000,3.500000e+01,35.000000,3.500000e+01,35.000000,3.500000e+01
mean,0.455730,4.663760e-02,0.340630,2.246040e-02,0.523924,4.050649e-02
std,0.417109,1.922997e-01,0.387901,9.330072e-02,0.238404,1.505412e-01
min,-0.497148,0.000000e+00,-0.636375,1.246242e-112,-0.038067,1.481650e-133
25%,0.288153,3.258743e-125,0.185869,7.125230e-78,0.365187,5.127194e-74
50%,0.528440,3.065543e-43,0.444237,8.008983e-34,0.583649,4.436513e-45
75%,0.766735,3.674005e-20,0.649873,1.212952e-09,0.730494,2.190610e-12
max,0.974928,9.360466e-01,0.817776,5.033092e-01,0.904865,7.967023e-01


### ¿Cuántos coeficientes hay en cada intervalo de valores?

In [6]:
ordenar_valores_cc (CoefCorrel, 'r_TD_Th', 'p_TD_Th', 'r_TD_Te', 'p_TD_Te', 'r_Th_Te', 'p_Th_Te')

,r_TD_Th,r_TD_Te,r_Th_Te
cc>0.5,18,16,23
cc>0,10,13,10
cc>-0.5,7,4,2
cc>-1,0,2,0
total,35,35,35


# 2.2) ¿Las oscilaciones diarias vienen del exterior? Coef a partir de las temperaturas suavizadas

### Preparar datos:

En el proceso de obtención de los coeficientes se borran datos. Es necesario volver a calcular las listas de dataframes por semanas cada vez que se quieran obtener coeficientes para evitar perder datos.

In [7]:
#Obtener los datos por semanas Cruce:
#2021
SemanaCruceJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx")
SemanaCruceAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")
SemanaCruceOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx")
#2022
SemanaCruceJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx")
SemanaCruceAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx")
SemanaCruceSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx")

#Obtener los datos por semanas Exterior:
#2021
SemanaExteriorJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto21 = SepararPorSemanas( "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre21 = SepararPorSemanas( "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")
SemanaExteriorOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx")
#2022
SemanaExteriorJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx")
SemanaExteriorAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx")
SemanaExteriorSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx")

#Obtener los datos por semanas Hall:
#2021
SemanaHallJulio21 = SepararPorSemanas("./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx")
SemanaHallAgosto21 = SepararPorSemanas("./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx")
SemanaHallSeptiembre21 = SepararPorSemanas("./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx")
SemanaHallOctubre21 = SepararPorSemanas("./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx")
#2022
SemanaHallJulio22 = SepararPorSemanas("./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx")
SemanaHallAgosto22 = SepararPorSemanas("./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx")
SemanaHallSeptiembre22 = SepararPorSemanas("./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx")

### Obtener coeficientes:

In [8]:
def cc_Th_Tc_Te(DataframeCruce, DataframeExterior, DataframeHall):#Función adaptada para variables suavizadas
    """
    Atributos:
    1) DataframeCruce: Lista de dataframes por semanas correspondiente a los datos de un mes de Cruce
    2) DataframeExterior: Lista de dataframes por semanas correspondiente a los datos de un mes de Exterior
    3) DataframeHall: Lista de dataframes por semanas correspondiente a los datos de un mes de Hall
    
    Devuelve:
    Una lista de listas, r. Cada elemento de r, r[i] con i=0, 1, 2, 3, 4, es una lista con los coeficiente de correlación y p 
    valores correspondientes a una semana.  r[i] = r_TD_Th, p_TD_Th, r_TD_Te, p_TD_Te, r_Th_Te, p_Th_Te
    
    ¿A qué corresponde cada elemento de r[i]?
    1) r_TD_Th: Coeficiente de correlación entre la temperatura del Cruce y del Hall
    2) p_TD_Th: P-valor correspondiente a r_TD_Th
    3) r_TD_Te: Coeficiente de correlación entre la temperatura del Cruce y la exterior
    4) p_TD_Te: P-valor correspondiente a r_TD_Te
    5) r_Th_Te: Coeficiente de correlación entre la temperatura del Hall y la exterior
    6) p_Th_Te: P-valor correspondiente a r_Th_Te
    """
    i=0
    r=[]
    while i<len(DataframeCruce):
       #print(i)
        DataframeCruce[i].reset_index(drop=True, inplace=True)
        DataframeExterior[i].reset_index(drop=True, inplace=True)
        DataframeHall[i].reset_index(drop=True, inplace=True)
        
        # Variables que me van a hacer falta: Tc, Te, Th
        for index, _ in DataframeExterior[i].iterrows():#Se va fuera de indice
            if  __builtins__.sum([pd.isna(DataframeCruce[i]['Tc_suavizada'][index]),
                                  pd.isna(DataframeHall[i]['Th_suavizada'][index]),
                                  pd.isna(DataframeExterior[i]['Te_suavizada'][index])]) > 0:
                DataframeCruce[i].drop([index], axis=0, inplace=True)
                DataframeExterior[i].drop([index], axis=0, inplace=True)
                DataframeHall[i].drop([index], axis=0, inplace=True)

        #Mediante scipy por semanas:
        r_TD_Th, p_TD_Th = stats.pearsonr(DataframeCruce[i]['Tc_suavizada'], DataframeHall[i]['Th_suavizada'])
        r_TD_Te, p_TD_Te = stats.pearsonr(DataframeCruce[i]['Tc_suavizada'], DataframeExterior[i]['Te_suavizada'])
        r_Th_Te, p_Th_Te = stats.pearsonr(DataframeHall[i]['Th_suavizada'], DataframeExterior[i]['Te_suavizada'])
        rs = r_TD_Th, p_TD_Th, r_TD_Te, p_TD_Te, r_Th_Te, p_Th_Te
        r.append(rs)
        i=i+1
    return(r)

In [9]:
#Obtener las correlaciones entre la temperatura del Cruce, la exterior y la del Hall:
#2021
r7_2021 = cc_Th_Tc_Te(SemanaCruceJulio21, SemanaExteriorJulio21, SemanaHallJulio21)
r8_2021 = cc_Th_Tc_Te(SemanaCruceAgosto21, SemanaExteriorAgosto21, SemanaHallAgosto21)
r9_2021 = cc_Th_Tc_Te(SemanaCruceSeptiembre21, SemanaExteriorSeptiembre21, SemanaHallSeptiembre21)
r10_2021 = cc_Th_Tc_Te(SemanaCruceOctubre21, SemanaExteriorOctubre21, SemanaHallOctubre21)
#2022
r7_2022 = cc_Th_Tc_Te(SemanaCruceJulio22, SemanaExteriorJulio22, SemanaHallJulio22)     
r8_2022 = cc_Th_Tc_Te(SemanaCruceAgosto22, SemanaExteriorAgosto22, SemanaHallAgosto22)
r9_2022 = cc_Th_Tc_Te(SemanaCruceSeptiembre22, SemanaExteriorSeptiembre22, SemanaHallSeptiembre22)

In [10]:
#Representar los valores de los coeficientes y p-valores en una tabla:
s1=pd.Series([r7_2021[0][0],r7_2021[1][0], r7_2021[2][0],r7_2021[3][0],r7_2021[4][0],
              r8_2021[0][0],r8_2021[1][0], r8_2021[2][0],r8_2021[3][0],r8_2021[4][0],#semanas de agosto 21
              r9_2021[0][0],r9_2021[1][0],r9_2021[2][0],r9_2021[3][0],r9_2021[4][0],#semanas de septiembre 21
              r10_2021[0][0],r10_2021[1][0],r10_2021[2][0],r10_2021[3][0],r10_2021[4][0],
              r7_2022[0][0], r7_2022[1][0], r7_2022[2][0], r7_2022[3][0], r7_2022[4][0],  
              r8_2022[0][0], r8_2022[1][0], r8_2022[2][0], r8_2022[3][0], r8_2022[4][0],
              r9_2022[0][0], r9_2022[1][0], r9_2022[2][0], r9_2022[3][0], r9_2022[4][0]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s2=pd.Series([r7_2021[0][1],r7_2021[1][1], r7_2021[2][1],r7_2021[3][1],r7_2021[4][1],
              r8_2021[0][1],r8_2021[1][1], r8_2021[2][1],r8_2021[3][1],r8_2021[4][1],#semanas de agosto 21
              r9_2021[0][1],r9_2021[1][1],r9_2021[2][1],r9_2021[3][1],r9_2021[4][1],#semanas de septiembre 21
              r10_2021[0][1],r10_2021[1][1],r10_2021[2][1],r10_2021[3][1],r10_2021[4][1],
              r7_2022[0][1], r7_2022[1][1], r7_2022[2][1], r7_2022[3][1], r7_2022[4][1],  
              r8_2022[0][1], r8_2022[1][1], r8_2022[2][1], r8_2022[3][1], r8_2022[4][1],
              r9_2022[0][1], r9_2022[1][1], r9_2022[2][1], r9_2022[3][1], r9_2022[4][1]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s3=pd.Series([r7_2021[0][2],r7_2021[1][2], r7_2021[2][2],r7_2021[3][2],r7_2021[4][2],
              r8_2021[0][2],r8_2021[1][2], r8_2021[2][2],r8_2021[3][2],r8_2021[4][2],#semanas de agosto 21
              r9_2021[0][2],r9_2021[1][2],r9_2021[2][2],r9_2021[3][2],r9_2021[4][2],#semanas de septiembre 21
              r10_2021[0][2],r10_2021[1][2],r10_2021[2][2],r10_2021[3][2],r10_2021[4][2],
              r7_2022[0][2], r7_2022[1][2], r7_2022[2][2], r7_2022[3][2], r7_2022[4][2], 
              r8_2022[0][2], r8_2022[1][2], r8_2022[2][2], r8_2022[3][2], r8_2022[4][2],
              r9_2022[0][2], r9_2022[1][2], r9_2022[2][2], r9_2022[3][2], r9_2022[4][2]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s4=pd.Series([r7_2021[0][3],r7_2021[1][3], r7_2021[2][3],r7_2021[3][3],r7_2021[4][3],
              r8_2021[0][3],r8_2021[1][3], r8_2021[2][3],r8_2021[3][3],r8_2021[4][3],#semanas de agosto 21
              r9_2021[0][3],r9_2021[1][3],r9_2021[2][3],r9_2021[3][3],r9_2021[4][3],
              r10_2021[0][3],r10_2021[1][3], r10_2021[2][3],r10_2021[3][3],r10_2021[4][3],
              r7_2022[0][3], r7_2022[1][3], r7_2022[2][3], r7_2022[3][3], r7_2022[4][3],
              r8_2022[0][3], r8_2022[1][3], r8_2022[2][3], r8_2022[3][3], r8_2022[4][3],
              r9_2022[0][3], r9_2022[1][3], r9_2022[2][3], r9_2022[3][3], r9_2022[4][3]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s5=pd.Series([r7_2021[0][4],r7_2021[1][4], r7_2021[2][4],r7_2021[3][4],r7_2021[4][4],
              r8_2021[0][4],r8_2021[1][4], r8_2021[2][4],r8_2021[3][4],r8_2021[4][4],#semanas de agosto 21
              r9_2021[0][4],r9_2021[1][4],r9_2021[2][4],r9_2021[3][4],r9_2021[4][4],#semanas de septiembre 21
              r10_2021[0][4],r10_2021[1][4],r10_2021[2][4],r10_2021[3][4],r10_2021[4][4],
              r7_2022[0][4], r7_2022[1][4], r7_2022[2][4], r7_2022[3][4], r7_2022[4][4], 
              r8_2022[0][4], r8_2022[1][4], r8_2022[2][4], r8_2022[3][4], r8_2022[4][4],
              r9_2022[0][4], r9_2022[1][4], r9_2022[2][4], r9_2022[3][4], r9_2022[4][4]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])
s6=pd.Series([r7_2021[0][5],r7_2021[1][5], r7_2021[2][5],r7_2021[3][5],r7_2021[4][5],
              r8_2021[0][5],r8_2021[1][5], r8_2021[2][5],r8_2021[3][5],r8_2021[4][5],#semanas de agosto 21
              r9_2021[0][5],r9_2021[1][5],r9_2021[2][5],r9_2021[3][5],r9_2021[4][5],#semanas de septiembre 21
              r10_2021[0][5],r10_2021[1][5],r10_2021[2][5],r10_2021[3][5],r10_2021[4][5],
              r7_2022[0][5], r7_2022[1][5], r7_2022[2][5], r7_2022[3][5], r7_2022[4][5],
              r8_2022[0][5], r8_2022[1][5], r8_2022[2][5], r8_2022[3][5], r8_2022[4][5],
              r9_2022[0][5], r9_2022[1][5], r9_2022[2][5], r9_2022[3][5], r9_2022[4][5]],
          index = ['Sem1 Julio 2021','Sem2 Julio 2021','Sem3 Julio 2021','Sem4 Julio 2021','Sem5 Julio 2021',
                   'Sem1 Agosto 2021','Sem2 Agosto 2021','Sem3 Agosto 2021','Sem4 Agosto 2021','Sem5 Agosto 2021',
                   'Sem1 Septiembre 2021','Sem2 Septiembre 2021','Sem3 Septiembre 2021','Sem4 Septiembre 2021',
                   'Sem5 Septiembre 2021',
                   'Sem1 Octubre 2021','Sem2 Octubre 2021','Sem3 Octubre 2021','Sem4 Octubre 2021','Sem5 Octubre 2021',
                   'Sem1 Julio 2022', 'Sem2 Julio 2022','Sem3 Julio 2022','Sem4 Julio 2022','Sem5 Julio 2022',
                   'Sem1 Agosto 2022','Sem2 Agosto 2022','Sem3 Agosto 2022','Sem4 Agosto 2022','Sem5 Agosto 2022',
                   'Sem1 Septiembre 2022','Sem2 Septiembre 2022','Sem3 Septiembre 2022','Sem4 Septiembre 2022',
                   'Sem5 Septiembre 2022'])

#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['r_TD_Th', 'p_TD_Th', 'r_TD_Te', 'p_TD_Te', 'r_Th_Te', 'p_Th_Te']
display(CoefCorrel)
CoefCorrel.describe()

,r_TD_Th,p_TD_Th,r_TD_Te,p_TD_Te,r_Th_Te,p_Th_Te
Sem1 Julio 2021,-0.159547,1.834706e-03,-0.546704,6.372187e-31,0.741487,2.563574e-67
Sem2 Julio 2021,-0.545575,2.215135e-53,-0.709943,4.145792e-104,0.457349,4.834339e-36
Sem3 Julio 2021,-0.438797,5.355849e-33,-0.570587,2.661395e-59,0.759385,3.523039e-127
Sem4 Julio 2021,0.470568,2.498902e-38,0.265943,2.421144e-12,0.183953,1.580772e-06
Sem5 Julio 2021,0.509890,1.924565e-39,-0.311850,1.859748e-14,0.243548,3.183187e-09
Sem1 Agosto 2021,0.009165,9.312866e-01,-0.103446,3.291690e-01,0.921808,2.145064e-38
Sem2 Agosto 2021,0.812108,6.751893e-159,0.104935,6.475287e-03,0.382050,8.865232e-25
Sem3 Agosto 2021,0.896878,8.413864e-238,0.484344,1.577684e-40,0.450335,1.273745e-34
Sem4 Agosto 2021,0.798310,1.084268e-149,0.695604,2.637864e-98,0.786329,2.913838e-142
Sem5 Agosto 2021,0.691023,9.819060e-48,0.724649,1.697812e-54,0.643179,1.437175e-39


,r_TD_Th,p_TD_Th,r_TD_Te,p_TD_Te,r_Th_Te,p_Th_Te
count,35.000000,3.500000e+01,35.000000,3.500000e+01,35.000000,3.500000e+01
mean,0.482008,2.701911e-02,0.345721,9.734274e-03,0.544470,3.162425e-02
std,0.472556,1.573587e-01,0.453469,5.559857e-02,0.241561,1.352297e-01
min,-0.561458,0.000000e+00,-0.759211,2.224198e-136,-0.053527,2.913838e-142
25%,0.303843,4.700135e-159,0.200509,1.225007e-91,0.394860,1.384170e-80
50%,0.645391,2.215135e-53,0.484344,1.577684e-40,0.605591,3.718497e-46
75%,0.818514,1.275643e-26,0.698729,5.864188e-12,0.740555,1.529578e-15
max,0.985313,9.312866e-01,0.848187,3.291690e-01,0.921808,7.758838e-01


### ¿Cuántos coeficientes hay en cada intervalo de valores?

In [12]:
ordenar_valores_cc (CoefCorrel, 'r_TD_Th', 'p_TD_Th', 'r_TD_Te', 'p_TD_Te', 'r_Th_Te', 'p_Th_Te')

,r_TD_Th,r_TD_Te,r_Th_Te
cc>0.5,20,17,23
cc>0,8,12,10
cc>-0.5,5,2,2
cc>-1,2,4,0
total,35,35,35
